# Adapting RED for demosaicing

## overview

RED assumes a forward model given by 

$$
\DeclareMathOperator{\vec}{vec}
\newcommand{\by}{\mathbf{y}}
\newcommand{\bx}{\mathbf{x}}
\newcommand{\bH}{\mathbf{H}}
\newcommand{\be}{\mathbf{e}}
\newcommand{\bi}{\mathbf{i}}
\newcommand{\bW}{\mathbf{W}}
\newcommand{\bB}{\mathbf{B}}
\newcommand{\bI}{\mathbf{I}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\by = \bH\bx + \be
$$

where $\by$ is the _degraded_ image, the result of a linear degradation operator $\bH$ applied to the original image $\bx$ under the influence of some noise $\be \sim \mathcal{N}(\mathbf{0},\sigma^2 \mathbf{I})$. 

RED tries to solve image inverse problems, i.e. acquiring $\bx$ given the forward model and $\by$, by introducing a regularization term $\rho$ that is dependent on a denoiser $\bf$ satisfying some weak constraints,

$$
\rho(\bx) = \frac{1}{2} \bx^T (\bx - f(\bx))
$$

which could be interpreted as unnormalized cross-correlation between $\bx$ and the denoising residual. $\rho(\bx)$ is small when either
- $\bx$ serves as fixed-point of the denoising engine, $\bx \approx f(\bx)$
- cross-correlation of residual to the image itself is small, i.e. orthogonal to each other. 
    - This prior is motivated from the rationale that noisy image can be thought of as orthogonal protrusions relative to a manifold of noiseless images

RED model the inverse problem as solution to the following optimization problem

$$
\hat{\bx} = \underset{\bx}{\text{argmin }} \frac{1}{2\sigma^2} \norm{\bH\bx - \by}_2^2 + \lambda\rho(\bx)
$$

RED solves the problem with 
- fixed-point method
- steepest-descent
- ADMM

Specifically, RED uses the following denoiser
- __median filter__ and 
- __Non-locally Centralized Sparse Representation (NCSR)__ 
    - trained specifically for images with images contaminated by white Gaussian noise with a fixed noise-level of $\sigma=5$
    
Note, RED's result is equiavlent to those obtained using the $P^3$ framework. However, parameter tuning in RED is a lot simpler.


## RED on C2B images

Note, the RED paper follows the procedure outlined in the NCSR paper for testing performance of their method on image restoration tasks. Specifically, there are points that need to be changed for the purpose of demosaicing C2B images
- additive gaussian noise added to the degraded image for all restoration tasks
    - set $\sigma=1$, apply minimal amount noise
- same with NCSR paper, restoring RGB image done by converting to YCbCr color-space, applying reconstruction algorithm on the luminance channel only, and then converting the result back to RGB domain
    - suspect, RED just follows NCSR's procedure. Guessing it might not work well on RGB color-space, i.e. ratios R/G, B/G not maintined explicitly by the denoising prior $\rho(\bx)$. However, for structured light C2B images, there is no explicit ratio relationship between channels: each channel is equally important.
    - Anyhow, just try the method on images on RGB color-space first and go from there.
- NCSR is trained in a supervised manner, specifically on images contaminated with white Gaussian noise with a fixed noise-level
    - may not be ideal when applied directly to C2B images: might need retraining
    - be good to try median filter denoiser as well
- NCSR denoiser is applied to luminance channel only
    - just use the denoiser on each channel independently
    
    
## Represent spatial subsampling as linear operator $\bH$

- done, next step try to write a function, that given any pattern assignment, gives the corresponding linear operator

## Problems

- the resulting demosaiced image $x$ from RED+ADMM showed mosaiced pattern
    - observed that regularization term drop very rapidly in a few iterations
    - probably because setting $x_i=0$ for as many $i$ as possible is effective in reducing the value of the regularization term
    - the reason that other reconstruction tasks such as superresolution does not experience similar problem is, i think, due to reason that each output pixel depends on 47 different input pixels whereas in demosaicing, each output pixel depends on 1 input pixel. For demosaicing, about 2/3 of pixels can be changed without affecting the $\by$ under $\bH$. These pixels could be safely set to 0 to reduce the value of regularization term. Same could not be said with other reconstruction deblurring
    - try decreasing value of $\lambda$: does not work, psnr still drops
    - try incoporate the multiplexing matrix $\bW$ into degradation operator $\bH$, in hope that more input pixels is required to compute 1 output pixel
    - try 2 pixel neighborhood
    

## Incorporating multiplexing matrix $\bW$ in degradation operator $\bH$

Consider 

$$
\by = \bH\bx + \be
$$

We have a vectorized linear operator $\bH = \bB (\bW \otimes \bI_P)$

$$
\vec(\by) = \bB \vec(\bx \bW^T) = \bB (\bW \otimes \bI_P) \vec(\bx)
\quad\text{where}\quad
\bB = 
\begin{pmatrix}
    \bH' & \mathbf{0} \\
    \mathbf{0} & \bH' \\
\end{pmatrix}
$$

where $\bH'$ is the demosaicing linear operator used previously.

We can compare the pattern of matrix $\bH'$ and $\bH$ for $S=3,P=4$ over 2x2 image

$$
\bH' = 
\begin{pmatrix}
    1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
    0& 0& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0 \\
    0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 0& 0 \\
    0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 1 \\
\end{pmatrix}
$$

$$
\bH = 
\begin{pmatrix}
    1& 0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0 \\
    0& 1& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
    0& 0& 1& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
    0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 1 \\
    0& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0 \\
    0& 0& 0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 1& 0& 0 \\
    0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 1& 0 \\
    0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 1& 0& 0& 0& 0 \\
\end{pmatrix}
$$

In the end, we try to infer $P\times S$ matrix $\bx$ given $P\times 2$ two-bucket measurements. Both of these data are readily available


